## Creation of the four PPS monomers

In [1]:
import mbuild as mb
import math
import gsd.hoomd
import numpy as np

# Creates a mBuild compound of a single PPS monomer
pps = mb.load("c1ccc(S)cc1", smiles=True)

In [2]:
#Removes all of they hydogens from the simulation
pps.remove(pps.particles_by_element('H'))

In [3]:
system = mb.compound.Compound()

In [4]:
#Creats a box with four monomers inside of it
for i in range(4):
    clone = mb.clone(pps)
    if i %2 == 0:
        clone.rotate(theta = i*(math.pi/2), around = [0, 0, 1])
    else:
        clone.rotate(theta = i*(math.pi/2), around = [0, 1, 0])
    clone.translate([(i + 1), 1, 1])
    system.add(clone)
system.box = mb.Box([6, 2, 2])

In [5]:
#Todo:
#Make a function that takes a gsd file and calculates vectors of orientation for gay-berne major axis given the pps
# positions

In [6]:
#The system of four monomers of PPS
system.save("pps_box.gsd", overwrite=True)
system.visualize()

/Users/noah/miniconda3/envs/try2/lib/python3.10/site-packages/gsd/hoomd.py:420: FutureWarning: Snapshot is deprecated, use Frame
  warnings.warn("Snapshot is deprecated, use Frame", FutureWarning)
/Users/noah/miniconda3/envs/try2/lib/python3.10/site-packages/gsd/hoomd.py:1125: FutureWarning: The 'wb' mode is deprecated, use 'w'
  gsdfileobj = fl.open(name=str(name),


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
file = gsd.hoomd.open("pps_box.gsd", "r")

In [8]:
frame = file[0]

In [9]:
frame.particles.position

array([[-2.1692837e+01,  1.6220557e-02, -2.0739976e-01],
       [-2.1020956e+01, -1.2000172e+00, -4.7733255e-02],
       [-1.9646132e+01, -1.2118286e+00,  2.1015719e-01],
       [-1.8941286e+01, -5.6079682e-03,  3.0836880e-01],
       [-1.7182194e+01, -1.9554812e-02,  6.3843250e-01],
       [-1.9615372e+01,  1.2115655e+00,  1.4862752e-01],
       [-2.0990221e+01,  1.2215596e+00, -1.0921624e-01],
       [-1.0207399e+01,  1.6220557e-02,  1.6928360e+00],
       [-1.0047733e+01, -1.2000172e+00,  1.0209563e+00],
       [-9.7898426e+00, -1.2118286e+00, -3.5386935e-01],
       [-9.6916313e+00, -5.6079682e-03, -1.0587134e+00],
       [-9.3615675e+00, -1.9554812e-02, -2.8178067e+00],
       [-9.8513727e+00,  1.2115655e+00, -3.8462818e-01],
       [-1.0109217e+01,  1.2215596e+00,  9.9022084e-01],
       [ 1.6928360e+00, -1.6220557e-02, -2.0739976e-01],
       [ 1.0209563e+00,  1.2000172e+00, -4.7733255e-02],
       [-3.5386935e-01,  1.2118286e+00,  2.1015719e-01],
       [-1.0587134e+00,  5.6079

## Getting the positions of the particles in one monomer

In [10]:
pps.save("one_pps.gsd", overwrite=True)

In [11]:
file = gsd.hoomd.open("one_pps.gsd", "r")
#Finds the real positions of the atoms in the monomer.
frame = file[0]
#partpos = particle position
partpos = frame.particles.position
#partmass = particle mass
partmass = frame.particles.mass

In [12]:
print(partpos)
len(partpos)
print(partmass)

[[-1.692836    0.01622056 -0.20739976]
 [-1.0209563  -1.2000172  -0.04773325]
 [ 0.35386935 -1.2118286   0.21015719]
 [ 1.0587134  -0.00560797  0.3083688 ]
 [ 2.8178067  -0.01955481  0.6384325 ]
 [ 0.38462818  1.2115655   0.14862752]
 [-0.99022084  1.2215596  -0.10921624]]
[12.011 12.011 12.011 12.011 32.06  12.011 12.011]


In [13]:
#Find the Greatest Distance between all the pairs of positions, then convert that difference between the positions
# as the vector function

In [31]:
maxvect = None
maxdist = 0
for i, vect0 in enumerate(partpos):
    for vect1 in partpos[i+1:]:
        vect = vect0 - vect1
        dist = np.linalg.norm(vect)
        if dist > maxdist:
            maxdist = dist
            maxvect = vect
print("The maximum distance of your monomer is", maxdist)
print("The vector that belongs to the maximum distance is", maxvect)

The maximum distance of your monomer is 4.589402
The vector that belongs to the maximum distance is [-4.510643    0.03577537 -0.8458322 ]


In [33]:
#Finds the center of mass of the monomer
M = np.sum(partmass)
posCoMnumerator = partpos * np.reshape(partmass, [np.shape(partpos)[0], 1])
CoMcords = (np.sum(posCoMnumerator, axis = 0)) / M
print(CoMcords)

[ 0.6476411  -0.00234209  0.2314996 ]


In [19]:
#Convert the vector along the major axis at the center of mass and convert that to quaternions


In [20]:
#Come back later and put this in a for loop for all of the frames

In [ ]:
#distanceList = []
#vectorList = []
#vectorcompDist = []
#for i in range(len(partpos)):
#    x0 = partpos[i][0]
#    y0 = partpos[i][1]
#    z0 = partpos[i][2]
    #print(x0, y0, z0) 
#    for j in range(len(partpos)):
#        xd = x0 - partpos[j][0]
#        yd = y0 - partpos[j][1]
#        zd = z0 - partpos[j][2]
#        vectorList.append([xd, yd, zd])
#        dist = np.sqrt((xd**2)+(yd**2)+(zd**2))
#        distanceList.append(dist)
#        vectorcompDist.append([partpos[i], partpos[j], dist])
#maxdist = max(distanceList)
#print("The maximum distance of your monomer is", maxdist)
#maxvect = max(vectorList)
#print("The vector that belongs to the maximum distance is", maxvect)
#normvect = maxvect / np.linalg.norm(maxvect)
#print("The unit vector along the major axis is :", normvect)

#whichelementVmax = vectorList.index(maxvect)
#whichelementDmax = distanceList.index(maxdist)

#for k in range(len(vectorcompDist)):
#    if vectorcompDist[k][2] == maxdist:
#        print("Our component vectors and distance are as follows:", vectorcompDist[k])

In [ ]:
#Finds the center of mass of the monomer
#xi_mi_positions = []
#yi_mi_positions = []
#zi_mi_positions = []
#for i in range(len(partpos)):
#    xi_mi = partpos[i][0] * partmass[i]
#    yi_mi = partpos[i][1] * partmass[i]
#    zi_mi = partpos[i][2] * partmass[i]
#    xi_mi_positions.append(xi_mi)
#    yi_mi_positions.append(yi_mi)
#    zi_mi_positions.append(zi_mi)
#M = np.sum(partmass)
#x_CoM = (np.sum(xi_mi_positions)) / M
#y_CoM = (np.sum(yi_mi_positions)) / M
#z_CoM = (np.sum(zi_mi_positions)) / M
#CoMcords = np.array([x_CoM, y_CoM, z_CoM])
#print("The center of Mass of your monomer is",CoMcords)